In [96]:
SHOW_OUTPUTS = True

def ShowTableau(c, a, b, certificate, operations, gap_vars):
    print("c[]: {}".format(c))
    print("A[] : {}".format(a))
    print("b[] : {}".format(b))

def identity(n):
    m=[[0 for x in range(n)] for y in range(n)]
    for i in range(0,n):
        m[i][i] = 1
    return m

def SimplexTableau(r, v, c, a, b):
    certificate = [0] * r
    operations = identity(3)
    gap_vars = identity(3)
    a = a + gap_vars
    c = c + [0] * r
    for i in range(len(c)):
        c[i] = c[i] * -1
        
    for c_index in range(len(c)):
        
        if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, operations, gap_vars)
        
        if(c[c_index] < 0 ):
            lower = 10000
            for i in range(len(b)):
                if(b[i][0] != 0):
                    if(SHOW_OUTPUTS): print("Minimizing: {}/{}".format(b[i][0],a[i][c_index]))
                    if(a[i][c_index] != 0 and b[j][0]/a[i][c_index] < lower):
                        lower = b[i][0]/a[i][c_index]
                        which_row = i
            if(SHOW_OUTPUTS): print("Lower value is {}".format(lower))        

        

In [97]:
r = 3
v = 3
c = [2,4,8]
a = [[1,0,0],[0,1,0],[0,0,1]]
b = [[1],[1],[1]]
SimplexTableau(r,v,c,a,b)

c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
b[] : [[1], [1], [1]]
Minimizing: 1/1
Minimizing: 1/0
Minimizing: 1/0
Lower value is 1.0
c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
b[] : [[1], [1], [1]]
Minimizing: 1/0
Minimizing: 1/1
Minimizing: 1/0
Lower value is 1.0
c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
b[] : [[1], [1], [1]]
Minimizing: 1/0
Minimizing: 1/0
Minimizing: 1/1
Lower value is 1.0
c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
b[] : [[1], [1], [1]]
c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
b[] : [[1], [1], [1]]
c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
b[] : [[1], [1], [1]]


In [75]:
print(range(len(a)))

range(0, 4)
